In [14]:
import numpy as np
from docx import Document
from docx.shared import Pt

In [15]:
f='gstr3b'
fname=f'{f}.docx'

doc=Document(fname)
all_tables=[]
for table in doc.tables:
    rows=len(table.rows)
    cols=len(table.columns)
    data=np.empty((rows,cols), dtype=object)
    for i,row in enumerate(table.rows):
        for j,cell in enumerate(row.cells):
            data[i,j]=cell.text.strip()
    all_tables.append(data)

In [16]:
l=['3.1 Details of Outward supplies and inward supplies liable to reverse charge (other than those covered by Table 3.1.1)',
'3.1.1 Details of Supplies notified under section 9(5) of the CGST Act, 2017 and corresponding provisions in IGST/UTGST/ SGST Acts',
'3.2 Out of supplies made in 3.1 (a) and 3.1.1 (i), details of inter-state supplies made',
'4. Eligible ITC',
'4. Eligible ITC',
'5 Values of exempt, nil-rated and non-GST inward supplies',
'5.1 Interest and Late fee for previous tax period',
'6.1 Payment of tax',
'Breakup of tax liability declared (for interest computation)']	

In [17]:
def update_document(row, col, new_value, table_index):
    if row >= len(doc.tables[table_index].rows) or col >= len(doc.tables[table_index].columns):
        print("Warning: The specified cell does not exist in the table.")
        return    
    all_tables[table_index][row, col] = new_value
    cell = doc.tables[table_index].rows[row].cells[col]
    cell.text = str(new_value)
    for paragraph in cell.paragraphs:
        for run in paragraph.runs:
            run.font.name, run.font.size = 'Roboto', Pt(7.5)

In [18]:
all_tables[6]=np.vstack(([all_tables[5][0]], all_tables[6]))
for k in range(len(l)):
    x=all_tables[k+2]
    y=np.array([l[k]]*(x.shape[1]))
    all_tables[k+2]=np.vstack((y, x))

In [19]:
# k=[]

# x=all_tables[j]
# for row_index, row in enumerate(x):
#     for col_index, value in enumerate(row):
#         if value=='0.00':
#             k.append((row_index-2,col_index,j))
                
# for i in k:
#     update_document(i[0], i[1], i, i[2])

# doc.save(f"{f}_Updated.docx")

In [20]:
lists=[]
for i in range(2):
    for row_index, row in enumerate(all_tables[i]):
        for col_index, value in enumerate(row):
            if len(value)==0:
                d={}
                d['Description']=''
                d['Row']=all_tables[i][row_index][-2]
                d['Value']='NaN'
                d['Column']=''
                d['Coordinates']=(row_index,col_index,i)
                lists.append(d)

In [21]:
for table_index in range(2,len(all_tables)):
    x=all_tables[table_index]
    for row_index, row in enumerate(x):
        for col_index, value in enumerate(row):
            if value=='0.00':
                d={}
                d['Description']=x[0,0]
                # d['Row']=f"{x[1,0]} | {x[row_index,0]}"
                d['Row']=x[row_index,0]
                d['Column']=x[1,col_index]
                d['Value']='NaN'
                if table_index==6:
                    d['Coordinates']=(row_index-2,col_index,table_index)
                else:
                    d['Coordinates']=(row_index-1,col_index,table_index)
                lists.append(d)

In [22]:
# o={}
# o['Total taxable value']='Value'
# o['Integrated tax']='IGST'
# o['Central tax']='CGST'
# o['State/UT tax']='SGST'
# o['State/ UT tax']='SGST'
# o['Cess']='Cess'
# o['']=''

# k=[f]
# for i_index, i in enumerate(lists):
#     if i['Column'] in o.keys() and i['Row'] in o.keys():
#         s=' | '.join([f"{i_index}.",o[i['Row']],o[i['Column']],i['Description']])
#     elif i['Row'] in o.keys():
#         s=' | '.join([f"{i_index}.",o[i['Row']],i['Column'],i['Description']])
#     elif i['Column'] in o.keys():
#         s=' | '.join([f"{i_index}.",i['Row'],o[i['Column']],i['Description']])
#     else:
#         s=' | '.join([f"{i_index}.",i['Row'],i['Column'],i['Description']])
#     k.append(s)
# k

In [23]:
o={}
o['Total taxable value']='Value'
o['Integrated tax']='IGST'
o['Central tax']='CGST'
o['State/UT tax']='SGST'
o['State/ UT tax']='SGST'
o['Cess']='Cess'
o['']=''

k=[]
for i_index, i in enumerate(lists):
    if i['Column'] in o.keys() and i['Row'] in o.keys():
        s=' | '.join([o[i['Row']],o[i['Column']],i['Description']])
    elif i['Row'] in o.keys():
        s=' | '.join([o[i['Row']],i['Column'],i['Description']])
    elif i['Column'] in o.keys():
        s=' | '.join([i['Row'],o[i['Column']],i['Description']])
    else:
        s=' | '.join([i['Row'],i['Column'],i['Description']])
    k.append(tuple((i_index,i['Coordinates'],s)))
k

[(0, (0, 1, 0), 'Year |  | '),
 (1, (1, 1, 0), 'Period |  | '),
 (2, (0, 1, 1), '1. GSTIN |  | '),
 (3, (1, 1, 1), '2(a). Legal name of the registered person |  | '),
 (4, (2, 1, 1), '2(b). Trade name, if any |  | '),
 (5, (3, 1, 1), '2(c). ARN |  | '),
 (6, (4, 1, 1), '2(d). Date of ARN |  | '),
 (7,
  (1, 1, 2),
  '(a) Outward taxable supplies (other than zero rated, nil rated and exempted) | Value | 3.1 Details of Outward supplies and inward supplies liable to reverse charge (other than those covered by Table 3.1.1)'),
 (8,
  (1, 2, 2),
  '(a) Outward taxable supplies (other than zero rated, nil rated and exempted) | IGST | 3.1 Details of Outward supplies and inward supplies liable to reverse charge (other than those covered by Table 3.1.1)'),
 (9,
  (1, 3, 2),
  '(a) Outward taxable supplies (other than zero rated, nil rated and exempted) | CGST | 3.1 Details of Outward supplies and inward supplies liable to reverse charge (other than those covered by Table 3.1.1)'),
 (10,
  (1, 4,

In [24]:
import pickle
with open('gstr3b_params.pkl', 'wb') as f1:
    pickle.dump(k, f1)

In [25]:
j=0
for i in lists:
    update_document(i['Coordinates'][0], i['Coordinates'][1], j, i['Coordinates'][2])
    j+=1

In [26]:
doc.save(f"{f}_Updated.docx")

In [27]:
import docx2pdf

In [28]:
docx2pdf.convert(f"{f}_Updated.docx", f"{f}.pdf")

  0%|          | 0/1 [00:00<?, ?it/s]